# Data Cleaning

This notebook details the process carried to obtain the datasets 'yb50.csv' and 'ts50.csv' which are used in the Business-and-Weather project.

The core business data is obtained from the [Yelp Dataset](https://www.yelp.com/dataset). Yelp data spans several states; below we will focus on data from Pittsburgh, Pennsylvania. 

Weather data from a land-based station in the Pittsburgh International Airport was obtained from the [National Centers for Environmental Information](https://www.ncdc.noaa.gov/data-access/).

The data is used here for personal and educational purposes.

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import json

from datetime import timedelta, date

We start by loading the datasets:

In [2]:
# read .json files into lists
data_b = [json.loads(line) for line in open('yelp_dataset/business.json','r')]
data_c = [json.loads(line) for line in open('yelp_dataset/checkin.json','r')]

# use pandas to build dataframes
df_b = pd.DataFrame(data_b) # for business info
df_c = pd.DataFrame(data_c) # for check-in info
df_w = pd.read_csv('weather_dataset/weather_data.csv',dtype=object) # for weather data

FileNotFoundError: [Errno 2] No such file or directory: 'yelp_dataset/business.json'

### Business data processing

Lets process the business and check_in datasets to exact the information most relevant to this project: business names, categories, and daily check-in counts (for businesses in Pittburgh).

In [ ]:
# Get Pittburg business data, and sort by business_id:
todrop = ['state','attributes','city','postal_code','latitude','longitude','is_open','hours','address','review_count','stars']
b_pit = df_b[df_b['city'].values=='Pittsburgh'].sort_values(by=['business_id'])

# Drop irrelevant columns:
b_pit.drop(todrop,axis=1,inplace=True)

# Get Pittburg check-in data for only those businesses in b_pit, and sort by business_id:
b_ids = b_pit['business_id']
c_pit = df_c[df_c['business_id'].isin(b_ids.tolist())].sort_values(by=['business_id'])

# Rename column named 'date' to 'check_in'.
c_pit.rename({'date':'check_in'}, axis=1,inplace=True)

# Merge the b_pit and c_pit into a single dataframe:
df = pd.merge(b_pit,c_pit, on= 'business_id')

# Clean up business indices
df['business_id'] = np.arange(0,len(df.index))
df = df.set_index(['business_id'])

# Add aggregate check-in count column
df['total_count'] = df['check_in'].str.split(', ').apply(len)

Exploring the data reveals that there are in fact duplicates of certain business names like 'Starbucks', which have stores different locations. We want to combine these into a single entry under the same name, aggregating information about check-in counts.   

In [ ]:
# Merge the check-in datetime events
def combine_ci(x):
    ci_list = []
    for elem in x:
        ci_list.extend(str.split(elem,', '))
        
    return ci_list    

# Group by common names and then aggregate by 
# - selecting the smallest business id and the longest cateory description 
# - combining the check-in events lists and summing their total count.
df = df.reset_index().groupby('name').agg({'business_id': 'min' ,'categories': lambda x : max(x.astype(str), key=len),'check_in' : combine_ci,'total_count': 'sum'})

# Restate index
df.sort_values(by='business_id',inplace=True)#.set_index(['business_id'])
df = df.reset_index().set_index(['business_id'])

 We shall pick the 50 businesses with highest total check-in counts. 

In [ ]:
# Select top businesses by total number of check-ins.

top = 50 # number of businesses we shall select

# Get the indices of the 50 top businesses
top_ids = df['total_count'].sort_values(ascending=False).index.values[:top] 

# Select out top businesses
df50 = df[df.index.isin(top_ids)].reset_index(drop=True)

print(df50.head())

We reduce the data to a particular range of dates: Jan 2014 through Dec 2018. 

In [ ]:
# Create iterable range of dates.
def daterange(start_date, end_date): 
    diff = end_date - start_date 
    for n in range(int(diff.days)): 
        yield start_date + timedelta(days = n)

# Rounds datetime to the nearest day
def round_day(t):
    return t.replace(second=0, minute=0,hour=0)
        
# Convert datetime string into datetime (to the nearest day).
def convert(sdt):
    date = round_day(dt.datetime.strptime(sdt, '%Y-%m-%d %H:%M:%S'))
    return date

# Convert list of datetime strings into a list of datetimes (to the nearest day).
def convert_list(ci_list):
    dates_list = [round_day(dt.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')) for date in ci_list]
    return dates_list

# Count the number of occurences of a single datetime inside a datetime list.
def count(dt_list, sdt = None):
    return dt_list.count(sdt) 
        
# Set datetime range
start_datetime = dt.datetime(2015, 1, 1) 
end_datetime = dt.datetime(2019, 1, 1) 

# Construct a new column for every daily count of check-ins
ts1 = (df50['check_in'].apply(convert_list)) # auxiliary function outside loop for efficiency
for single_datetime in daterange(start_datetime, end_datetime): 
    datestamp = single_datetime.date()
    df50[datestamp] = ts1.apply(count, sdt = single_datetime) 

# Obtain time-indexed daily check-in count across the 50 businesses
ts1 = df50.drop(['name','categories','check_in','total_count'],axis=1).T ;
ts1.index.name = 'Date'

ts1.head()

### Weather data processing

In [ ]:
# Clean up their datetime notation:
def nicedt(uglydt):
    return ' '.join(uglydt.split('T'))

# Pick Columns of Interest
ts2 = df_w[['DATE','HourlyDryBulbTemperature','HourlyPrecipitation','HourlyWindSpeed']].copy()

# Get clear datetime notation and type
ts2['DATE'] = pd.to_datetime(ts2['DATE'].apply(nicedt))

# Give cleaner names to the variables
ts2.columns = ['Date', 'T', 'P','WS']

# Round the hours 
ts2['Date'] = ts2['Date'].apply(round_day)

# Select the date range
ts2 = ts2[ (ts2['Date'] >= start_datetime) & (ts2['Date'] < end_datetime)]

# Clean weather data. There are extra notational 's' in the table to emphasize uncertain measurements. 
# We will include values even if only approximate.

ts2['T'] = ts2['T'].str.strip('s')
ts2['P'] = ts2['P'].str.strip('s')
ts2['WS'] = ts2['WS'].str.strip('s')

# Set trace precipitation values to zero
ts2['P'].replace('T','0.00',inplace=True)


## Fill Missing Reported Datetimes ##

# Replace * missing values
ts2['T'].replace('*',np.NaN,inplace=True)

# Get set of unique dates : for loop below
date_set = set(ts2['Date']) 

# Set the date as index
ts2 = ts2.set_index('Date') 

counter = 0
for datetime in daterange(start_datetime,end_datetime):
    if datetime in date_set:
        pass
    else:    
        counter += 1
        ts2 = ts2.append(pd.Series(name=datetime)) # add the datetime
        ts2 = ts2.sort_index() # sort the datetimes
        #print(datetime, 'was missing and was added with forward filling') # forward filling below

print('missing hours:',counter)

#Turn into numeric type and fill in missing values (forward/backward filling)
ts2 = ts2.astype(float)
ts2 = ts2.fillna(method ='ffill')
ts2 = ts2.fillna(method ='bfill')

# Data has different sources and report types, we will average duplicate datetimes.

# Use groupby and mean to remove duplicate dates.
ts2 = ts2.reset_index().groupby(['Date']).mean().reset_index()

# Check no redundant temperatures 
print('# idx values:',len(set(df_w.index.values)),'# of indices:', len(df_w.index))

# Round values
ts2[['T','WS']] = ts2[['T','WS']].round(1)
ts2['P'] = ts2['P'].round(2)

ts2 = ts2.set_index('Date')

ts2.head()

### Join business and weather frames 

In [ ]:
print('business days =', len(ts1.index))
print('weather days =', len(ts2.index))

In [ ]:
ts50 = ts1.join(ts2)
ts50.head()

### Write datasets to csv files

In [ ]:
df50[['name','categories']].to_csv('yb50.csv')
ts50.to_csv('ts50.csv')